In [1]:
import keras
import numpy as np

path = keras.utils.get_file(fname='nietzsche.txt',
                            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()

Using TensorFlow backend.


In [2]:
print('The length of corpus =', len(text))

The length of corpus = 600893


In [3]:
maxlen = 60
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

In [4]:
print('The number of sequences =', len(sentences))

The number of sequences = 200278


In [5]:
chars = sorted(list(set(text)))
print('The number of unique characters =', len(chars))

for i, c in enumerate(chars):
    print(repr('index = %d, character = %c' % (i, c)))

The number of unique characters = 57
'index = 0, character = \n'
'index = 1, character =  '
'index = 2, character = !'
'index = 3, character = "'
"index = 4, character = '"
'index = 5, character = ('
'index = 6, character = )'
'index = 7, character = ,'
'index = 8, character = -'
'index = 9, character = .'
'index = 10, character = 0'
'index = 11, character = 1'
'index = 12, character = 2'
'index = 13, character = 3'
'index = 14, character = 4'
'index = 15, character = 5'
'index = 16, character = 6'
'index = 17, character = 7'
'index = 18, character = 8'
'index = 19, character = 9'
'index = 20, character = :'
'index = 21, character = ;'
'index = 22, character = ='
'index = 23, character = ?'
'index = 24, character = ['
'index = 25, character = ]'
'index = 26, character = _'
'index = 27, character = a'
'index = 28, character = b'
'index = 29, character = c'
'index = 30, character = d'
'index = 31, character = e'
'index = 32, character = f'
'index = 33, character = g'
'index = 34, charact

In [6]:
char_index = dict((c, idx) for idx, c in enumerate(chars))
print('character index =', char_index)

character index = {'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}


In [7]:
# Vectorization
n_samples = len(sentences)
n_unique_chars = len(chars)
# One hot encoding
samples = np.zeros((n_samples, maxlen, n_unique_chars), dtype=np.bool)
targets = np.zeros((n_samples, n_unique_chars), dtype=np.bool)

for n, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        samples[n, t, char_index[char]] = 1
    targets[n, char_index[next_chars[n]]] = 1

In [8]:
from keras import models, layers

# Using recurrent layer or 1D convolutional layer to extract features from sequential samples.
model = keras.models.Sequential()
model.add(layers.LSTM(units=128, input_shape=(maxlen, n_unique_chars)))
model.add(layers.Dense(units=n_unique_chars, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [9]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.01),
              loss='categorical_crossentropy')

In [10]:
def sampling_procedure(preds, temperature=0.1):
    # Scaling probability distribution.
    preds = np.array(preds).astype('float64')
    log_preds = np.log(preds) / temperature
    exp_preds = np.exp(log_preds)
    # Make sure the sum of distribution is 1.
    preds = exp_preds / np.sum(exp_preds)
    # Take exprements by given probability.
    probs = np.random.multinomial(n=1, pvals=preds, size=1)
    # And return a most frequently choosed.
    return np.argmax(probs)

In [11]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text)-maxlen-1)

for epoch in range(1, 60):
    print('*' * 80)
    print('epoch =', epoch)
    model.fit(samples, targets, batch_size=128, epochs=1)

    seed_text = text[start_index:start_index+maxlen]
    print('seed text =', seed_text)

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('-' * 80)
        print('temperator=', temperature)

        generated_text = seed_text
        msg = generated_text
        
        for n in range(400):
            sampled = np.zeros((1, maxlen, n_unique_chars))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_index[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sampling_procedure(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            msg += next_char
            generated_text = generated_text[1:]
        if epoch >= 20:
            print(msg)

********************************************************************************
epoch = 1
Epoch 1/1
200278/200278 [==============================] - 86s 432us/step - loss: 1.9842
seed text = the slowly ascending ranks and classes, in which,
through fo
--------------------------------------------------------------------------------
temperator= 0.2
--------------------------------------------------------------------------------
temperator= 0.5
--------------------------------------------------------------------------------
temperator= 1.0
--------------------------------------------------------------------------------
temperator= 1.2
********************************************************************************
epoch = 2
Epoch 1/1
200278/200278 [==============================] - 86s 431us/step - loss: 1.6284
seed text = the slowly ascending ranks and classes, in which,
through fo
--------------------------------------------------------------------------------
temperator= 0.2
---------

/home/guhwanbae/anaconda3/envs/gu-keras/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


--------------------------------------------------------------------------------
temperator= 1.0
--------------------------------------------------------------------------------
temperator= 1.2
********************************************************************************
epoch = 14
Epoch 1/1
200278/200278 [==============================] - 81s 404us/step - loss: 1.3629
seed text = the slowly ascending ranks and classes, in which,
through fo
--------------------------------------------------------------------------------
temperator= 0.2
--------------------------------------------------------------------------------
temperator= 0.5
--------------------------------------------------------------------------------
temperator= 1.0
--------------------------------------------------------------------------------
temperator= 1.2
********************************************************************************
epoch = 15
Epoch 1/1
200278/200278 [==============================] - 81s 404us/ste

the slowly ascending ranks and classes, in which,
through for, men tone mances:
pujbroue 
take you
have haby ammurm
forearies in the sense.;--if me                    can someoccets of idesses is asfure because a metaphysical
guiged without
sluositard just estem different and med. inselves in tancty found and antiwnefily of limition carrios
of the adqual
takend often
begur as an aperi? were cares and new ages, to "stiputionity, immoral
tray, but irreating,
********************************************************************************
epoch = 22
Epoch 1/1
200278/200278 [==============================] - 81s 404us/step - loss: 1.3333
seed text = the slowly ascending ranks and classes, in which,
through fo
--------------------------------------------------------------------------------
temperator= 0.2
the slowly ascending ranks and classes, in which,
through for the sense of the strength in the sense of the strength in the strength and perhaps the strength and strict and strange, and al

the slowly ascending ranks and classes, in which,
through for the possible and case and the sense of the spirit of the present and the sense of the desire to the possible to the promise of the demanded to the substinuth and the generate the profound the interritation of the most promise of the promise of the promise of the present of the present and the subject the strongest would be any of the stronger of the subject that the same as a possible and the be
--------------------------------------------------------------------------------
temperator= 0.5
the slowly ascending ranks and classes, in which,
through for one for the master of the generate him also the summ superioristic men of the religious and power, his sense of the cause that they reason for the bad interpretation of the again to different that it is also a profound, as the attain to the constantly it would not remain reason, the science, the most of the barbarity of the worsh the discondition and discovered also it little s

...

the slowly ascending ranks and classes, in which,
through formul, the fundamentyl. every also europeans,
explains of thing the gwe. the feeling soked, the practic made at materine) (in the hand its former of a
strangt: one is almost no more netelfen strict with prevails stiff alledian the unnecemen for which lecture and soughted. one of human to tree time structure, "that inverence, on the really more any
dangerous supers dwarpenness, of a dangers featurn 
--------------------------------------------------------------------------------
temperator= 1.2
the slowly ascending ranks and classes, in which,
through formaus--namely untaking (honoutifie enough. and
sego, but ethics," and and sernes as withoft, is already only that, takes dutes,
all regard to the incoincine, thinking is isuding luthow from about the opind of knowledge and duratly: merajucatt merajucligent contemuted also a videment, our left. any old tough-raring commemperness the iu", from a sweth mentanis; and lake feels, how 

200278/200278 [==============================] - 84s 420us/step - loss: 1.3306
seed text = the slowly ascending ranks and classes, in which,
through fo
--------------------------------------------------------------------------------
temperator= 0.2
the slowly ascending ranks and classes, in which,
through for the moral soul of the most stand the the sense of the most strong of the most and soul of the same to which the sense of the consequently the still and also as a man is a soul of the superioric and interestion of the most promised and the moral moral and the moral same than a man is also any sensus to the individual and the whole of the most destrumes and soul and any sensus the still the fact t
--------------------------------------------------------------------------------
temperator= 0.5
the slowly ascending ranks and classes, in which,
through formaur. the problem in his father for the art and are in such any soul only one of the
self-same and the superiorical false and the co

the slowly ascending ranks and classes, in which,
through for in the state of say that man warm, or a continue of his proper delitive the most draw and of the germans and the subject and are always but the last have the most developments of the most still at a philosophy does not difficult of every one which is delight of
the strange of such a man is a philosopher the still and has only in the strange to man are will be
superior the words
who are attain to
--------------------------------------------------------------------------------
temperator= 1.0
the slowly ascending ranks and classes, in which,
through forgoit with
oppers a mere an e by of mansox and higher!

philosian to rabliving, from than. he can couse
that it has upon the human "the symbwing and caller, better musugh
to facularly, from the tempo to coles how tim ake into them in general praiss, than argiss, ideas away of thing of him? that is master indivulsticitiatiane, that greet to mistather ogical, for the german and
sho

the slowly ascending ranks and classes, in which,
through for another, and "provited this manifested and understaned, generally transquence the threates have quality askgul for one should clauses misanded of a modern "its leakeded, to what
be nothing i tumber thus
car an things with a sho spothing of fite, i quiew! reality, in
authorh,
got, a lower man-futher.--which, controlative--if more either apprecers to-respreted to pullians which
would can
what latt
********************************************************************************
epoch = 56
Epoch 1/1
200278/200278 [==============================] - 82s 409us/step - loss: 1.3248
seed text = the slowly ascending ranks and classes, in which,
through fo
--------------------------------------------------------------------------------
temperator= 0.2
the slowly ascending ranks and classes, in which,
through formerly and the greatest the possible the fact that the sentiment of the comparency and soul of the power, and the compared and c

the slowly ascending ranks and classes, in which,
through for the suffers the sense of the most designated the sense of the world and soul, the most depressible the sense of the most conscience of the most such a soul and senses and the man is senses for the most problem of the sense of the sense of the sense of the propering the sense of the last for the sense of the world and the senses and senses of the good the senses of the propense of the propering t
--------------------------------------------------------------------------------
temperator= 0.5
the slowly ascending ranks and classes, in which,
through for the animal conceined ancient the last words, the more disguise and world the ascendaulitude, he who is a taste and plays from dirination the easily for a conscience it is will having the demicately himself of the sensed in the own the permitted the last higher of the experience of the concerning the senses and senses. the believed to will solemn experience of religious and scie

In [ ]:
model.save('nietzsche_language_model.h5')